# Big Data: Spark RDD

## Getting acquainted with Spark and Spark Notebook

 Never used a Notebook? 
 Find useful advice in the User Interface Tour (in the help menu) or in the 
 [Spark Notebook documentation](https://github.com/spark-notebook/spark-notebook/blob/master/docs/exploring_notebook.md) itself.
 

## Deeper Understanding of Spark

The goal of the next steps in the assignment is to gain a deeper understanding in what happens "inside".

Do not just click shift enter on every cell, but try to grasp what is happening by looking into the Spark UI after executing each command.

You should try to learn by using variants of the example queries; create a new cell, or duplicate the cell and modify it.

In [ ]:
import org.apache.spark.HashPartitioner

In [ ]:
val rddRange = sc.parallelize(0 to 999,8)

Remember that evaluation is lazy, and only happens upon actions, not transformations; i.e., so far, nothing happened. 

Check the Spark UI: jobs and stages do not yet contain entries corresponding to this command.

_Note: Port number :4040 can be higher, e.g., :4041, depending on how many notebooks you have running in the kernel._

In [ ]:
printf( "Number of partitions: %d\n", rddRange.partitions.length)
rddRange.partitioner

The default number of partitions depends on the number of cores in the machine that runs the docker engine; see e.g., `cat /proc/cpuinfo` from a `bash` shell in the docker engine.

The next command creates pairs of numbers, that we will treat as key-value pairs in the remainder of this notebook.

If you have not looked at Spark documentation so far, this would be a good time to go through the information on transformations and PairRDD functions:
- https://spark.apache.org/docs/2.3.2/rdd-programming-guide.html#transformations
- https://spark.apache.org/docs/2.3.2/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions

In [ ]:
val rddPairs = rddRange.map(x => (x % 100, 1000 - x))

In [ ]:
rddPairs.take(15)

In [ ]:
rddPairs.partitioner

As we see, the default way of processing does not assign a partitioner; the framework partitions the data in the default way, which is merely a guess. Several ways influence how partitioning takes place, where the easiest is to assign a partitioner.

In [ ]:
val rddPairsPart2 = rddPairs.partitionBy(new HashPartitioner(2))

In [ ]:
printf( "Number of partitions: %d\n", rddPairsPart2.partitions.length)
rddPairsPart2.partitioner

See how that influences processing; notice that `rddPairs` is partitioned in the default way, whereas `rddPairsPart2` uses only two partitions, based on our instructions.

In [ ]:
val rddPairsGroup = rddPairs.groupByKey()

In [ ]:
rddPairsGroup.toDebugString

In [ ]:
printf( "Number of partitions: %d\n", rddPairsGroup.partitions.length)
rddPairsGroup.partitioner

In [ ]:
rddPairsGroup.partitions.map(p => (p, p.index, p.hashCode))

In [ ]:
val rddPairsGroupPart2 = rddPairsPart2.groupByKey()

In [ ]:
rddPairsGroupPart2.partitions.map(p => (p, p.index, p.hashCode))

_Q: Spot the differences between the results, and try to map what you see on Chapter 2 that we read for the course._

Observe how the number of partitions of a groupByKey operation varies depending on the way the input is partitioned. This in turn affects the number of machines that will be at work in subsequent operations. Take a look at the Spark UI to see the difference for the two cases.

_Note: using explicit naming of RDDs helps you keep track of which job corresponds to which case._

In [ ]:
val rddPGP2Count = rddPairsGroupPart2.map( {case(x,y) => (x, y.reduce((a,b) => a + b))} )

In [ ]:
rddPGP2Count.name = "Partitioned Group Counts (2)"

In [ ]:
rddPGP2Count.take(10)

In [ ]:
rddPGP2Count.partitions.size

In [ ]:
rddPGP2Count.toDebugString

In [ ]:
printf( "Number of partitions: %d\n", rddPGP2Count.partitions.length)
rddPGP2Count.partitioner

Q: do you understand why the partitioner is none? (No worries if not, you will find a clue below.)

In [ ]:
val rddPairsPart4 = rddPairs.partitionBy(new HashPartitioner(4))

In [ ]:
rddPairsPart4.take(10)

In [ ]:
val rddA = rddPairsPart4.values.map( x => x  + 10 )

In [ ]:
printf( "Number of partitions: %d\n", rddA.partitions.length)
rddA.partitioner

In [ ]:
val rddB = rddPairsPart4.mapValues( x => x + 10 )

In [ ]:
printf( "Number of partitions: %d\n", rddB.partitions.length)
rddB.partitioner

_Q: Why are the results different for rddA and rddB? How is query processing affected by the partitioners?

Summarizing: partitioning depends on the distributed operations that are executed, and only operations with guarantees about the output distribution will carry an existing partitioner over to its result.

Another way to control the level of parallellism during query execution is to use the `repartition` and `coalesce` operations. 

In [ ]:
val rddC = rddA.repartition(2)

In [ ]:
rddC.partitions.map(p => (p, p.index, p.hashCode))

In [ ]:
rddC.toDebugString

In [ ]:
val rddD = rddB.coalesce(2)

Remember that we need actions for things to happen - so if you inspect the Spark UI, there are now no query plans corresponding to the above commands. Let us take two samples from the results. Do you understand the jobs and stages that you find in the Spark UI?

In [ ]:
rddC.takeSample(true, 10);

In [ ]:
rddD.takeSample(true, 10);

_Q: Compare the two query plans for `rddC` and `rddD`. Can you explain why the second query plan has on less shuffle phase?_

## Wrap up

If you have reached this point propery and understood what you observed, you have a solid understanding of Spark and its execution model.

Assignment 3B will move up the stack to consider the Dataframe API.